## <p style="text-align:center;color:midnightblue; font-size:190%;font-family:times new roman;"><u>Technical Data Science Program Curriculum Design<u\></p>

## <p style="text-align:center;color:black; font-size:150%;font-family:times new roman;">Master of Data Science and Analytics</p>

### <p style="color:black;font-family:times new roman;font-size:120%;"> Import Required Libriaries <p/>

In [65]:
#import required libraries
!pip install utils
!pip install normalization
import normalization
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import utils
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from pylab import *
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib.font_manager import FontProperties
import pandas as pd
import string
import nltk
from collections import Counter
import pandas as pd
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
from scipy import sparse

You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### <p style="color:black;font-family:times new roman;font-size:120%;"> Get required data </p>
<p style="color:black;font-family:times new roman;font-size:140%;"> A dataset of about 100 different technical courses (containing core & electives) was created.</p>

In [66]:
#Read Data from xlsx file 
tech_course_descr_df = pd.read_excel('University_Syllabus_desp.xlsx', index_col=None, encoding='utf8')

#Function to stem list 
def stem_list(_list):
    
    """This function stems the contents of a given list and returns 
    a new list with stemmed words """
    
    stemmed_list = []
    for i in range(len(_list)):
        tokenize_list = _list[i].split()
        temp1 =[]
        for j in range(len(tokenize_list)):
            _stem = porter.stem(tokenize_list[j])
            temp1.append(_stem)
            if len(temp1) > 1:
                temp2 = " ".join(temp1)
            else:
                temp2 = "".join(temp1)
        stemmed_list.append(temp2)
    return stemmed_list

### <p style="color:black;font-family:times new roman;font-size:120%;"> Preprocessing Data </p>
<p style="color:black;font-family:times new roman;font-size:140%;"> This data is extracted into a pandas data frame. The data is then cleaned by case conversion and stemming. </p>

In [67]:
#Clean Data
tech_course_descr_df['Course Descriptions'] = tech_course_descr_df.apply(lambda _: '', axis=1)
tech_course_descr_df['Course Descriptions'] = tech_course_descr_df['Course Description'].str.lower()


In [68]:
tech_course_descr_df['Course Descriptions'][1]

'an introduction to computer architecture and distributed systems with an emphasis on warehouse scale computing systems. topics will include fundamental tradeoffs in computer systems, hardware and software techniques for exploiting instruction-level parallelism, data-level parallelism and task level parallelism, scheduling, caching, prefetching, network and memory architecture, latency and throughput optimizations, specialization, and an introduction to programming data center computers.'

<p style="color:black;font-family:times new roman;font-size:140%;"> The following list is combination of two lists, first was from Kaggle data set and second, was from indeed job postings using TFID vectorizer. The combined list is used for further code. Both of these lists can be seen seperatey in the first "Job and skills cleaning" notebook<p/>

In [69]:
skills_list = ['identify data',
 'data management',
 'gradient boosting',
 'querying',
 'survival analysi',
 'cnn',
 'project management',
 'quantitative',
 'cross validation',
 'risk management',
 'data warehouse',
 'time series analysi',
 'bayesian technique',
 'mars',
 'unsupervised learning',
 'evolutionary approache',
 'ensemble method',
 'proprietary algorithm',
 'pca',
 'clustering',
 'strategic management',
 'tabular data',
 'gan',
 'link analysi',
 'support vector machine',
 'data analysi',
 'boosting',
 'genetic algorithm',
 'hidden markov models hmm',
 'monte carlo method',
 'factor analysi',
 'a/b testing',
 'rnn',
 'recommendation engine',
 'machine learning',
 'marketing',
 'consumer behavior',
 'finance',
 'bagging',
 'automation',
 'consulting',
 'collaborative filtering',
 'computer vision',
 'enterprise tool',
 'random forest',
 'kaggle ranking',
 'fraud detection',
 'agile methodology',
 'text analytic',
 'machine translation',
 'logistic regression',
 'financial engineering',
 'speech recognition',
 'hmm',
 'warehousing',
 'regression',
 'prescriptive modeling',
 'pattern recognition',
 'visualization',
 'effective communication',
 'markov logic networksnaive baye',
 'advertising',
 'business intelligence',
 'decision tree',
 'cloud computing',
 'data base',
 'predictive analytic',
 'genetic & evolutionary algorithm',
 'large scale data',
 'data structure',
 'adversarial learning',
 'mooc',
 'unstructured data',
 'data visualization',
 'relational database',
 'deep learning',
 'data engineering',
 'decision making',
 'structured data',
 'troubleshooting',
 'clear communication',
 'neural network',
 'evolutionary algorithm',
 'data architecture',
 'knn',
 'reinforcement learning',
 'segmentation',
 'accounting',
 'cluster analysi',
 'recommender system',
 'naive baye',
 'simulation',
 'digital marketing',
 'markov logic network',
 'social network analysi',
 'lift analysi',
 'rule induction',
 'feature engineering',
 'product design',
 'pca dimensionality reduction',
 'text mining',
 'neural net',
 'supervised machine learning',
 'outlier detection',
 'hidden markov model',
 'bayesian method',
 'uplift modeling',
 'data modeling',
 'svm',
 'association rule',
 'time serie',
 'dimensionality reduction',
 'pipeline',
 'natural language processing',
 'gradient boosted machine']
skills_list = list(set(skills_list))
len(skills_list)

115

<p style="color:black;font-family:times new roman;font-size:140%;"> Now, the skills are compared with each course description to extract the matched skills. <p/>

In [70]:
#Function to extract matched skills
def _extract_matched_skills(text, skills):
    match_skills =[]
    for p in range(len(skills)):
        if skills[p] in text:
            match_skills.append(skills[p])
    match_skills = list(set(match_skills))
    return match_skills


#Extracting matched skills from each course description
tech_course_descr_df['Matched Skills'] = tech_course_descr_df.apply(lambda _: '', axis=1)
for i in range(len(tech_course_descr_df['Course Descriptions'])):
    matched_skills = _extract_matched_skills(tech_course_descr_df['Course Descriptions'][i], skills_list)
    tech_course_descr_df['Matched Skills'][i] = matched_skills


<p style="color:black;font-family:times new roman;font-size:140%;"> The frequnecy of the matched skills is calculated. Subsequently, a matrix based on the skills list is created. <p/>

In [71]:
#Create new coloumn for the number of matched skills 
tech_course_descr_df['Matched Skills Frequency'] = tech_course_descr_df.apply(lambda _: '', axis=1)

#Filling the matched skills frequency column with the frequency of matched job skills 
for i in range(len(tech_course_descr_df['Matched Skills'])):
    matched_skills_frequency = len(tech_course_descr_df['Matched Skills'][i])
    tech_course_descr_df['Matched Skills Frequency'][i] = matched_skills_frequency

#Function to create Matrix 
def create_matrix_row(skills, matched_skills):
    matrix_row =[]
    for i in range(len(skills)):
        if skills[i] in matched_skills:
            matrix_row += [1]
        else: 
            matrix_row += [0]
    return matrix_row

#Create new coloumn
tech_course_descr_df['Matrix Row'] = tech_course_descr_df.apply(lambda _: '', axis=1)

#Add Matrix row to curresponding row of dataframe
for i in range(len(tech_course_descr_df['Matched Skills'])):
    matrix_row = create_matrix_row(skills_list, tech_course_descr_df['Matched Skills'][i])
    tech_course_descr_df['Matrix Row'][i] = matrix_row

<p style="color:black;font-family:times new roman;font-size:140%;"> The relative frequencies of each skill based on the job descriptions which was created earlier in 'Job and skills cleaning' is uploaded and extracted into a pandas data frame. Aditionally, a new matrix is created based on this relative frequency.<p/>

In [72]:
#Create new coloumn
tech_course_descr_df['Matched Skills Frequency'] = tech_course_descr_df.apply(lambda _: '', axis=1)


#Read Data from xlsx file 
rel_freq_df = pd.read_csv('Technical_relative_frequency.csv')

#Function to create relative frequnecy Matrix 
def create_rel_freq_matrix_row(skills, matched_skills, rel_freq):
    matrix_row =[]
    for i in range(len(skills)):
        if skills[i] in matched_skills:
            matrix_row += [rel_freq[i]]
        else: 
            matrix_row += [0]
    return matrix_row

#Create new coloumn
tech_course_descr_df['Relative Fequency Matrix Row'] = tech_course_descr_df.apply(lambda _: '', axis=1)


#Adding Matrix row to curresponding row of dataframe
for i in range(len(tech_course_descr_df['Matched Skills'])):
    matrix_row = create_rel_freq_matrix_row(rel_freq_df['list'], tech_course_descr_df['Matched Skills'][i], rel_freq_df['frequency'])
    tech_course_descr_df['Relative Fequency Matrix Row'][i] = matrix_row
    

<p style="color:black;font-family:times new roman;font-size:140%;"> A  <i> "Total Score" </i>  is given to each course based on the total relative frequency that each course has attained through matched skills. This completes the creation of the required dataframe. Analysis will be done using this dataframe. </p>

In [73]:
#Create new coloumn 
tech_course_descr_df['Total Course Score'] = tech_course_descr_df.apply(lambda _: '', axis=1)


#Adding total course score to corresponding course 
for i in range(len(tech_course_descr_df['Matched Skills'])):
    score = sum(tech_course_descr_df['Relative Fequency Matrix Row'][i])
    tech_course_descr_df['Total Course Score'][i] = score
    

tech_course_descr_df.head()

,University,Program Title,Courses,Course Type,Course Description,Course Descriptions,Matched Skills,Matched Skills Frequency,Matrix Row,Relative Fequency Matrix Row,Total Course Score
0,Columbia University,Masters in Data Science,Introduction to Data Science,Core,Data Science is a dynamic and fast growing fie...,data science is a dynamic and fast growing fie...,"[machine learning, quantitative, pipeline]",,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896
1,Columbia University,Masters in Data Science,Computer Systems for Data Science,Core,An introduction to computer architecture and d...,an introduction to computer architecture and d...,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,Columbia University,Masters in Data Science,Machine Learning for Data Science,Core,COMS 4721 is a graduate-level introduction to ...,coms 4721 is a graduate-level introduction to ...,"[supervised machine learning, machine learning]",,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0786526
3,Columbia University,Masters in Data Science,Algorithms for Data Science,Core,"Methods for organizing data, e.g. hashing, tre...","methods for organizing data, e.g. hashing, tre...","[gan, collaborative filtering, pca]",,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.110603
4,Columbia University,Masters in Data Science,Probability,Core,A calculus-based introduction to probability t...,a calculus-based introduction to probability t...,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [74]:
tech_course_descr_df['Matrix Row'] = tech_course_descr_df['Matrix Row'].apply(lambda x: np.array(x))

## we have initially created sparse 'feature matrix' using count vectorizer but we noticed a problem. for example we have a skill called 'data mining' the count vecotirizer is dividing this in to 'data' and 'mining' and due this the number of unique words obtained by count vectorizer is more than our inital list of algorithms. So we created our own sparse matrix using the colum 'Matrix Row'. Now the word 'data mining' is not sepearted during clustering and we cant get courses similar to each other on one cluster 

In [75]:
feature_matrix = np.array(tech_course_descr_df['Matrix Row'].tolist())

In [76]:
feature_matrix = sparse.csr_matrix(feature_matrix)

In [77]:
feature_matrix_dbscan = feature_matrix

### <p style="color:black;font-family:times new roman;font-size:120%;"> Application of Algorithm </p>
> #### <p style="color:black;font-family:times new roman;font-size:140%;"> Density Based Cluster Analysis <br> using DBSCAN (Density-Based Spatial Clustering of Applications with Noise) 

<p style="color:black;font-family:times new roman;font-size:140%;"> The text clustering
has an embedded process of unsupervised learning, where text group’s discovery
is performed without any prior information that could link each text to some
specific group </p> 

In [78]:
dbscan = DBSCAN(eps= 0.1, min_samples=2)

In [79]:
dbscan

DBSCAN(algorithm='auto', eps=0.1, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=1, p=None)

In [80]:
data = dbscan.fit(feature_matrix)
labels = data.labels_
#core_samples = np.zeros_like(labels, dtype = bool)
core_samples = data.core_sample_indices_
#core_samples[data.core_sample_indices_] = True

#Number of clusters formed 
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("Number of clusters formed:", n_clusters_)
core_samples

Number of clusters formed: 8


array([ 1,  2,  4,  5,  7,  8, 10, 11, 13, 16, 17, 29, 31, 38, 40, 46, 48,
       57, 62, 63, 66, 70, 71, 72, 73, 74, 75, 80, 82, 83, 84, 85, 86, 87,
       88, 89, 93], dtype=int64)

In [81]:
#clusters = [data[labels == i] for i in range(n_clusters_)]
clusters = []
for i in range(n_clusters_):
    temp = dbscan.labels_[i]
    clusters.append(temp)
print(clusters)


outliers = data.labels_[-1]
outliers

[-1, 0, 1, -1, 0, 2, -1, 2]


0

In [82]:
y_pred_dbscan = dbscan.labels_

In [83]:
y_pred_df_dbscan = pd.DataFrame(y_pred_dbscan)

In [84]:
cluster_counts_dbscan = pd.DataFrame(y_pred_df_dbscan[0].value_counts())

In [85]:
cluster_counts_dbscan 

,0
-1,57
0,22
2,3
1,2
7,2
6,2
5,2
4,2
3,2


Main drawback 
- cannot deal with varying densities 
- sensitive to parameters
- suffers from curse of dimensionality 


In [86]:
clustered_df_dbscan = tech_course_descr_df[['Courses','Matrix Row','Total Course Score','Matched Skills']]

In [87]:
clustered_df_dbscan["Cluster ID"] = y_pred_df_dbscan

C:\Users\Naga Raja Paidimarri\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [88]:
clustered_df_dbscan = clustered_df_dbscan.sort_values('Cluster ID')

In [89]:
clustered_df_dbscan

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
0,Introduction to Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896,"[machine learning, quantitative, pipeline]",-1
35,Regression Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.023509,"[mars, logistic regression, regression]",-1
36,Methods in Multivariate Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0883559,"[cluster analysi, factor analysi, machine lear...",-1
37,Unknown,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0198801,[project management],-1
78,statistical programming,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0886715,"[visualization, data analysi]",-1
39,Methods in Time Series Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0266646,"[time serie, regression]",-1
77,programming fundamentals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.121174,"[gan, structured data]",-1
41,Applied Analysis using SAS Enterprise Guide,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.194146,"[gan, quantitative, segmentation, marketing]",-1
42,Financial Accounting,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.155806,"[decision making, neural network, logistic reg...",-1
43,marketing engineering,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140265,"[gan, decision making, accounting]",-1


### As you can see the above data frame, except for the category '-1' rest all were perfectly clustered. Now we use Kmeans on the '-1' category courses to cluster them furthur. By this we are taking advantages of both Algorithms

In [90]:
clustered_df_kmeans = clustered_df_dbscan.sort_values('Cluster ID')

In [91]:
# clustered_df_kmeans.to_csv('clusteredtech.csv')

In [92]:
clustered_df_kmeans = clustered_df_kmeans.iloc[0:57,:]

In [93]:
clustered_df_kmeans['Matrix Row'] =clustered_df_kmeans['Matrix Row'].apply(lambda x: np.array(x))
feature_matrix = np.array(clustered_df_kmeans['Matrix Row'].tolist())
feature_matrix = sparse.csr_matrix(feature_matrix)

In [94]:
feature_matrix

<57x115 sparse matrix of type '<class 'numpy.int32'>'
	with 166 stored elements in Compressed Sparse Row format>

In [95]:
clustered_df_kmeans

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
0,Introduction to Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896,"[machine learning, quantitative, pipeline]",-1
33,Optimization & Heuristics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.174109,"[gan, finance, marketing]",-1
34,Social Networks Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00126223,"[recommender system, social network analysi]",-1
79,Data Visualisation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0402335,[visualization],-1
91,Natural language processing,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0114389,[natural language processing],-1
90,artificial intelligence,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.101452,"[decision making, machine learning]",-1
3,Algorithms for Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.110603,"[gan, collaborative filtering, pca]",-1
6,Exploratory Data Analysis & Visualization,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0732092,"[data visualization, visualization, clustering]",-1
9,Applied Machine Learning,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.097586,"[svm, neural network, gradient boosting, rando...",-1
12,NLP : Computational Models of Social Meaning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.120306,"[gan, natural language processing]",-1


In [96]:
for i in range(len(clustered_df_dbscan['Cluster ID'])):
    if clustered_df_dbscan['Matched Skills'][i] == []:
        print(clustered_df_dbscan['Courses'][i], [i])

Computer Systems for Data Science [1]
Probability [4]
Casual Inference for Data Science [10]
Elements of Data Science: First Course [11]
Projects in Data Science [13]
Business from Analytics in Digital Age [29]
Advanced programming using sas [40]
introduction to programming [46]
big data and NoSQL program [66]
Concepts of statistics(1) [73]
Concepts of statistics(2) [74]
 mathematics for data scientist [75]
large scale data storage systems [80]
introduction to computational biology [82]
datasystems in bioscience [83]
research in biotechnology [84]
data science project for life scientist [85]
data science project for computer scientist [86]
cyber security essentials [87]
pervasive application development [88]
encryption and authentication [89]
distributed computing systems [93]


> #### <p style="color:black;font-family:times new roman;font-size:140%;"> K-Means Cluster Analysis <br> using centroid based clustering  

The number of clusters is choosen as 15 because on average we expect to create a total of 15 courses for each circulum and for this we wanted to choose one course from each clsuter.

In [97]:
from sklearn.cluster import KMeans

# define the k-means clustering function
def k_means(feature_matrix, num_clusters):
    km = KMeans(n_clusters=num_clusters,max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters

# setting k value
num_clusters = 15

# get clusters and assigned the cluster labels to the movies
km_obj, clusters = k_means(feature_matrix=feature_matrix, num_clusters=num_clusters)
clustered_df_kmeans['Cluster'] = clusters

km_obj

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10000,
    n_clusters=15, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [98]:
from collections import Counter
# get the total number of courses per cluster
c = Counter(clusters)
print(c.items())

dict_items([(0, 1), (2, 9), (11, 2), (13, 7), (1, 17), (7, 3), (3, 1), (14, 3), (10, 1), (12, 1), (6, 5), (9, 1), (4, 2), (8, 3), (5, 1)])


In [99]:
clustered_df_kmeans = clustered_df_kmeans.sort_values('Cluster')

In [100]:
clustered_df_kmeans 

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID,Cluster
0,Introduction to Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896,"[machine learning, quantitative, pipeline]",-1,0
22,Business Communication and Analytics Consulting,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0851215,"[project management, finance, marketing]",-1,1
28,Deep Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0208268,"[computer vision, speech recognition, deep lea...",-1,1
69,data warehousing,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0723414,"[data architecture, warehousing, data warehouse]",-1,1
14,Big Data Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0250079,[finance],-1,1
19,Introduction to Databases and Information Retr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.0375513,"[data management, querying]",-1,1
64,computer vision,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00702114,[computer vision],-1,1
37,Unknown,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0198801,[project management],-1,1
15,Big Data in Finance,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0452035,"[risk management, structured data, unstructure...",-1,1
24,Analytics Value Chain,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,[a/b testing],-1,1


### Nowin the following code,The aim is to combine this clustered courses by kmeans with the dbscan data frame such that these kmeans data frame replaces the '-1' clustered category on the dbscan data frame. To make sure that the category numbers dont match, I multiplied the above kmeans cluster with -1 and conveted '0' to -15. 

In [101]:
clustered_df_kmeans['Cluster ID'] = clustered_df_kmeans['Cluster ID']*clustered_df_kmeans['Cluster']

In [102]:
clustered_df_kmeans

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID,Cluster
0,Introduction to Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896,"[machine learning, quantitative, pipeline]",0,0
22,Business Communication and Analytics Consulting,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0851215,"[project management, finance, marketing]",-1,1
28,Deep Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0208268,"[computer vision, speech recognition, deep lea...",-1,1
69,data warehousing,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0723414,"[data architecture, warehousing, data warehouse]",-1,1
14,Big Data Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0250079,[finance],-1,1
19,Introduction to Databases and Information Retr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0.0375513,"[data management, querying]",-1,1
64,computer vision,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00702114,[computer vision],-1,1
37,Unknown,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0198801,[project management],-1,1
15,Big Data in Finance,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0452035,"[risk management, structured data, unstructure...",-1,1
24,Analytics Value Chain,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,[a/b testing],-1,1


In [103]:
for i in range(len(clustered_df_kmeans)):
    if clustered_df_kmeans.iloc[i,4] == 0:
        clustered_df_kmeans.iloc[i,4] = -15

In [104]:
clustered_df_kmeans = clustered_df_kmeans.sort_values('Cluster ID')

In [105]:
clustered_df_kmeans = clustered_df_kmeans.drop('Cluster', axis=1)

In [106]:
# clustered_df_kmeans

In [107]:
clustered_df = clustered_df_dbscan

In [108]:
clustered_df

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
0,Introduction to Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896,"[machine learning, quantitative, pipeline]",-1
35,Regression Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.023509,"[mars, logistic regression, regression]",-1
36,Methods in Multivariate Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0883559,"[cluster analysi, factor analysi, machine lear...",-1
37,Unknown,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0198801,[project management],-1
78,statistical programming,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0886715,"[visualization, data analysi]",-1
39,Methods in Time Series Analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0266646,"[time serie, regression]",-1
77,programming fundamentals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.121174,"[gan, structured data]",-1
41,Applied Analysis using SAS Enterprise Guide,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.194146,"[gan, quantitative, segmentation, marketing]",-1
42,Financial Accounting,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.155806,"[decision making, neural network, logistic reg...",-1
43,marketing engineering,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140265,"[gan, decision making, accounting]",-1


In [109]:
clustered_df.iloc[0:57,:] = clustered_df_kmeans.iloc[0:57,:]

In [110]:
clustered_df = clustered_df.sort_values('Cluster ID')

In [111]:
clustered_df 

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
0,Introduction to Data Science,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.140896,"[machine learning, quantitative, pipeline]",-15
81,machine learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0112023,"[reinforcement learning, unsupervised learning...",-14
23,Data Mining,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0155412,"[predictive analytic, unsupervised learning]",-14
26,Analytics for Big Data,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0140423,[predictive analytic],-14
6,Exploratory Data Analysis & Visualization,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0732092,"[data visualization, visualization, clustering]",-13
59,intelligent information retrieval,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.16906,"[gan, visualization, natural language processi...",-13
52,Data Visualisation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0646892,"[data visualization, visualization]",-13
49,advanced data analysis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0732092,"[visualization, factor analysi, decision makin...",-13
50,python programming,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0.0859104,"[visualization, data management, data structure]",-13
79,Data Visualisation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0402335,[visualization],-13


In [112]:
clustered_df = clustered_df.drop('Matrix Row', axis=1)

In [113]:
# clustered_df_kmeans.sort_values('Cluster').to_csv('clusteredtech2.csv')

In [114]:
clustered_df.to_csv('clusteredtechcourses.csv')

### you can see the complete clustered courses in the 'clusteredtechcourses' file. There are around 20 clusters. Now we used a mix of Total score (based on relative frequency) and clustering to select courses for our circullum. our main is to pick courses from different clusters so that there wont be any repetition of algorithms in the circullum. But we also want to pick courses with high 'Total Course score' so that skills demanded in job market are covered.

In [115]:
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

#Get cluster details

def get_cluster_data_dbscan(cluster_data, num_clusters, topn_features=10):
    cluster_details = {}
    # get courses belonging to each cluster
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        courses = cluster_data[cluster_data['Cluster ID'] == cluster_num]['Courses'].values.tolist()
        cluster_details[cluster_num]['Courses'] = courses
    return cluster_details


#Print data
def print_cluster_data(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details:'.format(cluster_num))
        print('-'*20)
        print()
        print('Courses in this cluster:')
        print(', '.join(cluster_details['Courses']))
        print()
        print('='*100)
        print()
#get dbscan cluster details
dbscan_details = get_cluster_data_dbscan(clustered_df_dbscan, n_clusters_ , topn_features=10)
#print dbscan cluster details
print_cluster_data(dbscan_details)

def plot_cluster_plot_points_dbscan(num_clusters, feature_matrix, cluster_data, course_data):
    # build cosine distance matrix
    cosine_distance = 1 - cosine_similarity(feature_matrix)
    # dimensionality reduction using MDS
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
    # get coordinates of clusters in new low-dimensional space
    plot_positions = mds.fit_transform(cosine_distance)
    x_pos, y_pos = plot_positions[:, 0], plot_positions[:, 1]
    # map each unique cluster label with its coordinates and courses
    cluster_plot_df = pd.DataFrame({'x': x_pos,'y': y_pos,'label': course_data['Cluster ID'].values.tolist(),'Course': course_data['Courses'].values.tolist()})
    cluster_plot_df.to_csv('Technical_DBSCAN.csv')
    
    
#Get cluster details

def get_cluster_data_kmeans(cluster_data, num_clusters, topn_features=10):
    cluster_details = {}
    # get courses belonging to each cluster
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        courses = cluster_data[cluster_data['Cluster'] == cluster_num]['Courses'].values.tolist()
        cluster_details[cluster_num]['Courses'] = courses
    return cluster_details

def plot_cluster_plot_points_kmeans(num_clusters, feature_matrix, cluster_data, course_data):
    # build cosine distance matrix
    cosine_distance = 1 - cosine_similarity(feature_matrix)
    # dimensionality reduction using MDS
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
    # get coordinates of clusters in new low-dimensional space
    plot_positions = mds.fit_transform(cosine_distance)
    x_pos, y_pos = plot_positions[:, 0], plot_positions[:, 1]
    # map each unique cluster label with its coordinates and courses
    cluster_plot_df = pd.DataFrame({'x': x_pos,'y': y_pos,'label': course_data['Cluster'].values.tolist(),'Course': course_data['Courses'].values.tolist()})
    cluster_plot_df.to_csv('Technical_KMEANS.csv')

#get plotting points for dbscan
plot_cluster_plot_points_dbscan(n_clusters_, feature_matrix_dbscan, dbscan_details, clustered_df_dbscan)

#get kmeans cluster details
k_means_details = get_cluster_data_kmeans(clustered_df_kmeans, num_clusters, topn_features=10)

#print kmeans cluster details
print_cluster_data(dbscan_details)

#get plotting points for dbscan
plot_cluster_plot_points_kmeans(num_clusters, feature_matrix, k_means_details, clustered_df_kmeans)

Cluster 0 details:
--------------------

Courses in this cluster:
research in biotechnology, data science project for life scientist, datasystems in bioscience, data science project for computer scientist, cyber security essentials, pervasive application development, encryption and authentication, Concepts of statistics(2), Concepts of statistics(1), introduction to computational biology, large scale data storage systems,  mathematics for data scientist, introduction to programming, distributed computing systems, Computer Systems for Data Science, Probability, Advanced programming using sas, Casual Inference for Data Science, Business from Analytics in Digital Age, big data and NoSQL program, Elements of Data Science: First Course, Projects in Data Science


Cluster 1 details:
--------------------

Courses in this cluster:
Machine Learning for Data Science, Bioinformatics


Cluster 2 details:
--------------------

Courses in this cluster:
Probability & Statistics for Data Science, Stat

KeyError: 'Cluster'